In [2]:
!pip install langchain-google-genai # install the necessary module
!pip install gradio
!pip install gtts # Install the gtts package


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.0 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.17 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.6 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import tempfile
from gtts import gTTS
from langchain_google_genai import ChatGoogleGenerativeAI
from google.oauth2 import service_account
import os
import gradio as gr
from PIL import Image # Import the Image module


# Configuration
GEMINI_MODEL = "gemini-1.5-flash"
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
logo_path = "Lett.png"  # Replace with the actual path
LANGUAGE_CODES = {
    "Spanish": "es",
    "German": "de",
    "Romainian": "ro"
}

# Translation Logic
def translate_text(text, target_language):
    llm = ChatGoogleGenerativeAI(google_api_key=GEMINI_API_KEY, model=GEMINI_MODEL, temperature=0.3)
    prompt = f"Translate the following text: {target_language}: '{text}'. No styling and no extra text."
    translated_text = llm.invoke(prompt).content
    prompt2 = f"Give a translation for the following text: {target_language}: '{text}', and then give an detailed explanation for the translated words of each sentence. No styling."
    explanation = llm.invoke(prompt2).content
    return translated_text, explanation


# Text-to-Speech Logic
def generate_audio(text, target_language):
    language = LANGUAGE_CODES.get(target_language, "en")
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as audio_file:
        tts = gTTS(text=text, lang=language)
        tts.save(audio_file.name)
        audio_path = audio_file.name
    return audio_path

# Gradio Interface
def create_interface():
    with gr.Blocks(theme=gr.themes.Soft()) as demo:
      # Add headline and logo at the top

        with gr.Row():
          with gr.Column():
            gr.Markdown("# Hi, I’m Tabby! ")  # Headline using Markdown
            gr.Markdown("""
            **Powered by my advanced capabilities, this app offers seamless translation of large text
            chunks and entire documents with remarkable accuracy. Whether it's complex phrases or technical jargon, I
            ensure every translation retains the original meaning. With an easy-to-use interface and cutting-edge AI,
            it’s perfect for both personal and professional use.**
            """)
            gr.Markdown("""
            **Currently, we proudly support translations in Spanish, German, and Romanian — but hey,
            with a little bit of funding (hint, hint), we’ll be able to expand our language offerings
            and cover even more regions of the world. Stay tuned!**
            """)
          logo = Image.open(logo_path)
          gr.Image(value=logo, show_label=False, container=False)
        with gr.Row(equal_height=True):
            with gr.Column():  # Create a column for text input
                text_input = gr.Textbox(label="Enter text to translate", lines="8")
            with gr.Column():  # Create a column for file upload
                file_input = gr.File(label="Upload a document", type="filepath")
        with gr.Row():
            lang_dropdown = gr.Dropdown(list(LANGUAGE_CODES.keys()), label="Select Target Language")
        with gr.Row(visible=False) as output_row:
            translated_text = gr.Text(label="Translated Text", show_copy_button=True)
            audio_output = gr.Audio(label="Hear the translation", type="filepath")

        def on_submit(text, file, target_language):
            if file:
              try:
            # Attempt to open with UTF-8 first
                with open(file.name, "r", encoding="utf-8") as f:
                  text = f.read()
              except UnicodeDecodeError:
            # If UTF-8 fails, try another encoding (e.g., 'latin-1')
                with open(file.name, "r", encoding="latin-1") as f:
                  text = f.read()
            translated, explanation = translate_text(text, target_language)
            audio_path = generate_audio(translated, target_language)
            return {
                output_row: gr.update(visible=True),
                translated_text: explanation,
                audio_output: audio_path,
            }

        submit_btn = gr.Button("Translate")
        submit_btn.click(on_submit, inputs=[text_input, file_input, lang_dropdown],
                        outputs=[output_row, translated_text, audio_output])

    return demo

# Run the app
if __name__ == "__main__":
    demo = create_interface()
    demo.launch(share=True, debug=True)

ModuleNotFoundError: No module named 'torch'

In [32]:
print(gtts.lang.tts_langs())

{'af': 'Afrikaans', 'am': 'Amharic', 'ar': 'Arabic', 'bg': 'Bulgarian', 'bn': 'Bengali', 'bs': 'Bosnian', 'ca': 'Catalan', 'cs': 'Czech', 'cy': 'Welsh', 'da': 'Danish', 'de': 'German', 'el': 'Greek', 'en': 'English', 'es': 'Spanish', 'et': 'Estonian', 'eu': 'Basque', 'fi': 'Finnish', 'fr': 'French', 'fr-CA': 'French (Canada)', 'gl': 'Galician', 'gu': 'Gujarati', 'ha': 'Hausa', 'hi': 'Hindi', 'hr': 'Croatian', 'hu': 'Hungarian', 'id': 'Indonesian', 'is': 'Icelandic', 'it': 'Italian', 'iw': 'Hebrew', 'ja': 'Japanese', 'jw': 'Javanese', 'km': 'Khmer', 'kn': 'Kannada', 'ko': 'Korean', 'la': 'Latin', 'lt': 'Lithuanian', 'lv': 'Latvian', 'ml': 'Malayalam', 'mr': 'Marathi', 'ms': 'Malay', 'my': 'Myanmar (Burmese)', 'ne': 'Nepali', 'nl': 'Dutch', 'no': 'Norwegian', 'pa': 'Punjabi (Gurmukhi)', 'pl': 'Polish', 'pt': 'Portuguese (Brazil)', 'pt-PT': 'Portuguese (Portugal)', 'ro': 'Romanian', 'ru': 'Russian', 'si': 'Sinhala', 'sk': 'Slovak', 'sq': 'Albanian', 'sr': 'Serbian', 'su': 'Sundanese', 'sv